# XAI - TP2: Visualizations for Neural Networks
### Anne Gagneux


**Topic**:
Deep Learning models are considered “black box” models, i.e. we can't say much about how the neural network makes its prediction. The goal of this TP is to be able to determine, for a classification task, which parts of the image influence the prediction, in order to explain how the NN behave.

In [ ]:
# useful librairies
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt
import pickle
import urllib.request
import cv2
import numpy as np
from PIL import Image

%matplotlib inline


## Pixel attribution (Saliency Maps)

**Goal**: Highlight the pixels that were important in an image for the neural network prediction.

**How does it work?**: It is called a *gradient-based method*: we will compute the gradient of the prediction with respect to the input features (i.e. the pixels).
The general idea is that if a slight change in a pixel impacts a lot the prediction (i.e. if the aboluste value of the gradient with respect to this pixel is large), then this pixel is relevant for the prediction.

**Method**:
1. Compute the prediction (i.e. *the forward pass*).
2. Compute the gradient of the class score of interest with respect to the input pixels. The gradients are set to zero for all classes except the desired class, which is set to 1.
3. Vizualize the gradients.





### Download the Model
We provide you a pretrained model `ResNet-34` for `ImageNet` classification dataset.
* **ImageNet**: A large dataset of photographs with 1 000 classes.
* **ResNet-34**: A deep architecture for image classification.

In [ ]:
resnet34 = models.resnet34(pretrained=True)
resnet34.eval()  # set the model to evaluation mode

![ResNet34](https://miro.medium.com/max/1050/1*Y-u7dH4WC-dXyn9jOG4w0w.png)


Input image must be of size (3x224x224).

First convolution layer with maxpool.
Then 4 ResNet blocks.

Output of the last ResNet block is of size (512x7x7).

Average pooling is applied to this layer to have a 1D array of 512 features fed to a linear layer that outputs 1000 values (one for each class). No softmax is present in this case. We have already the raw class score!

In [ ]:
classes = pickle.load(urllib.request.urlopen(
    'https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl'))

# classes is a dictionary with the name of each class
print(classes)

### Input Images
We provide you 20 images from ImageNet (download link on the webpage of the course or download directly using the following command line,).<br>
In order to use the pretrained model resnet34, the input image should be normalized using `mean = [0.485, 0.456, 0.406]`, and `std = [0.229, 0.224, 0.225]`, and be resized as `(224, 224)`.

In [ ]:
def preprocess_image(dir_path):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    dataset = datasets.ImageFolder(dir_path, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),  # resize the image to 224x224
        transforms.ToTensor(),  # convert numpy.array to tensor
        normalize]))  # normalize the tensor

    return (dataset)

In [ ]:
# The images should be in a *sub*-folder of "data/" (ex: data/TP2_images/images.jpg) and *not* directly in "data/"!
# otherwise the function won't find them

import os
os.mkdir("data")
os.mkdir("data/TP2_images")
!cd data/TP2_images && wget "https://is1-ssl.mzstatic.com/image/thumb/Purple3/v4/50/0d/5d/500d5dad-2ddd-4556-4def-4b629a4b0ec0/source/256x256bb.jpg" && wget "https://www.strydomstud.com/s3.amazonaws.com/static.strydomstud.co.za/files/posts/339/images/medium3374.jpg"
dir_path = "data/"
dataset = preprocess_image(dir_path)

In [ ]:
# show the orignal image
index = 1
input_image = Image.open(dataset.imgs[index][0]).convert('RGB')
plt.imshow(input_image)

In [ ]:
output = resnet34(dataset[index][0].view(1, 3, 224, 224))
values, indices = torch.topk(output, 3)
print("Top 3-classes:", indices[0].numpy(),
      [classes[x] for x in indices[0].numpy()])
print("Raw class scores:", values[0].detach().numpy())

In [ ]:
def SaliencyMap( model, image, my_class_of_interest):
  model.eval() # make sure the model is in eval mode

  #we want to calculate gradient of higest score w.r.t. input
  #so set requires_grad to True for input
  image.requires_grad = True

  # compute the output (i.e. the prediction of the models)
  output = # TO COMPLETE

  # backpropagate to compute the gradient with respect to the input
  # TO COMPLETE

  saliency_map = # TO COMPLETE

  # renormalize between 0 and 1
  saliency_map = # TO COMPLETE
  return saliency_map

In [ ]:
my_image = dataset[-1][0].view(1, 3, 224, 224)
pixel_attrib_map = SaliencyMap(resnet34, my_image, 382)
print(pixel_attrib_map.shape)

In [ ]:
input_image = transforms.CenterCrop(224)(
    Image.open(dataset.imgs[index][0]).convert('RGB'))
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(input_image)
plt.xticks([])
plt.yticks([])
plt.subplot(1, 2, 2)
plt.imshow(pixel_attrib_map.detach().numpy(), cmap=plt.cm.hot)
plt.xticks([])
plt.yticks([])
plt.show()


## GradCAM

**Goal**: Highlight *regions* of pixels that were important in an image for the neural network prediction. The goal of GradCAM is to understand at which parts of an image a convolutional layer “looks” for a certain classification.

**How does it work?**: Here, the gradient is not backpropagated all the way back to the image, but to the last convolutional layer to produce a coarse localization map that highlights important regions of the image.

There are $k$ features maps in the last convolutional layer: $A_1, A_2, \dots, A_k$. Grad-CAM has to decide how important each of the $k$ feature map was to our class $c$ that we are interested in. We have to weight each pixel of each feature map with the gradient before we average over the feature maps.
We are looking for weights $\alpha^c_k$ that tell us how important is the pixel of each feature map, then we get the map by averaging the weighted features map:
$$map = ReLU \left(\sum_k \alpha^c_k A^k \right)$$

**Method**:
1. Compute the prediction (i.e. *the forward pass*).
2. Backpropagate the gradient of the *raw* class score of interest to the last convolution layer (before the FC layers). The gradients are set to zero for all classes except the desired class, which is set to 1.
3. Weight each feature map with:
$$\alpha_k^c = \frac 1 Z \sum_i \sum_j \frac{\partial y_{pred}}{\partial A^k_{i,j}}$$
4. Compute the heatmap:
$$map = ReLU \left(\sum_k \alpha^c_k A^k \right)$$
3. Scale the heatmap to the size of the input image.
4. Vizualize the gradients.

* **Hints**:
 + We need to record the output and grad_output of the feature maps to achieve Grad-CAM. In pytorch, the function `Hook` is defined for this purpose. Read the tutorial of [hook](https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html#forward-and-backward-function-hooks) carefully.
 + The size of feature maps is 7x7, so your heatmap will have the same size. You need to project the heatmap to the resized image (224x224, not the original one, before the normalization) to have a better observation. The function [`torch.nn.functional.interpolate`](https://pytorch.org/docs/stable/nn.functional.html?highlight=interpolate#torch.nn.functional.interpolate) may help.  

In [ ]:
def get_activations(mymodel, index, my_class_of_interest):

    mymodel.eval()

    activations = []
    activations_grad = []

    def forward_hook(layer, _, outputs_of_the_layer):
        # TO COMPLETE
        pass

    def backward_hook(layer, _, outputs_of_the_layer):
        # TO COMPLETE
        pass

    mymodel.layer4.register_forward_hook(forward_hook)
    mymodel.layer4.register_backward_hook(backward_hook)

    data = dataset[index][0].view(1, 3, 224, 224)
    result = mymodel(data)
    mymodel.zero_grad()

    result[:, my_class_of_interest].backward(retain_graph=True)

    return activations, activations_grad

In [ ]:
def alpha_k(activations_grad, k):
    return activations_grad[0][:, k, :, :].mean()

In [ ]:
def create_map(activations, activations_grad):

    n_feature_maps = activations[0].shape[1]
    width = activations[0].shape[2]
    alphas = [alpha_k(activations_grad,k) for k in range(n_feature_maps)]
    heat_map = torch.zeros((1,1,width, width))
    for k in range(n_feature_maps):
        heat_map += # TO COMPLETE
    heat_map = F.relu(heat_map)

    #rescale the map
    rescaled_heat_map = F.interpolate(heat_map, size = (224,224), mode = "bilinear")
    rescaled_heat_map /= rescaled_heat_map.max()
    return heat_map, rescaled_heat_map

In [ ]:
def Grad_CAM(index_img, my_class_of_interest):
    resnet34 = models.resnet34(pretrained=True)
    act, act_grad = get_activations(resnet34, index_img, my_class_of_interest)
    heat_map, rescaled_heat_map = create_map(act, act_grad)
    return heat_map, rescaled_heat_map

In [ ]:
index = 0


fig, axs = plt.subplots(1, 3, figsize=(20, 10))
output = resnet34(dataset[index][0].view(1, 3, 224, 224))
values, indices = torch.topk(output, 3)
print(indices[0])
top_class = indices[0][0]
top_classes = [classes[x] for x in indices[0].numpy()]
print(top_classes)
img = transforms.CenterCrop(224)(
    Image.open(dataset.imgs[index][0]).convert('RGB'))
axs[0].imshow(img, alpha=1)
axs[0].set_title('Picture {}'.format(index+1))

heat_map, rescaled_heat_map = Grad_CAM(index, top_class)
axs[1].imshow(heat_map[0, 0, :, :].detach().numpy())
axs[2].imshow(img, alpha=1)
axs[2].imshow(rescaled_heat_map.detach().numpy()[
              0, 0, :, :], cmap="jet", alpha=0.6)
axs[2].set_title("Class {}".format(top_classes[0]))

plt.show()

In [ ]:
index = 0

some_class = 282  # class : tiger cat

fig, axs = plt.subplots(1, 3, figsize=(20, 10))
output = resnet34(dataset[index][0].view(1, 3, 224, 224))

img = transforms.CenterCrop(224)(
    Image.open(dataset.imgs[index][0]).convert('RGB'))
axs[0].imshow(img, alpha=1)
axs[0].set_title('Picture {}'.format(index+1))

heat_map, rescaled_heat_map = Grad_CAM(index, some_class)
axs[1].imshow(heat_map[0, 0, :, :].detach().numpy())
axs[2].imshow(img, alpha=1)
axs[2].imshow(rescaled_heat_map.detach().numpy()[
              0, 0, :, :], cmap="jet", alpha=0.6)
axs[2].set_title("Class {}".format(classes[some_class]))

plt.show()